In [13]:
import requests, time
import pandas as pd
from tqdm import tqdm
from bs4 import BeautifulSoup
from selenium import webdriver
from datetime import datetime, timedelta

In [5]:
url = 'https://www.kr-weathernews.com/mv3/html/lifeindex.html?region='
header = {'User-Agent' : 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36'}
res = requests.get(url, headers=header)
res.encoding = 'utf-8'
res.text[:500]

'<!doctype html><html><head><title>생활지수</title><meta charset=utf-8><meta http-equiv=X-UA-Compatible content="IE=Edge"><meta name=theme-color content=#0c419a><meta name=viewport content="width=device-width,initial-scale=1,shrink-to-fit=no,maximum-scale=1,user-scalable=no"><meta name=title content=생활지수><meta name=description content="세계 최대의 기상 컨설팅 회사, 날씨, 주간날씨, 주말날씨, 기상 컨설팅, 해운 날씨 컨설팅, 항공 날씨 컨설팅"><meta property=og:url content="//www.kr-weathernews.com/mv3/html/main.html?region=1100000000"><meta pro'

In [6]:
from selenium import webdriver

driver = webdriver.Chrome()     # PATH 설정 할 것 (chromedriver.exe가 위치해있는곳을 path 설정해야함)
driver.get(url)

The chromedriver version (119.0.6045.105) detected in PATH at C:\Users\human\Downloads\chromedriver.exe might not be compatible with the detected chrome version (120.0.6099.71); currently, chromedriver 120.0.6099.71 is recommended for chrome 120.*, so it is advised to delete the driver in PATH and retry


TimeoutException: Message: timeout: Timed out receiving message from renderer: 299.972
  (Session info: chrome=120.0.6099.71)
Stacktrace:
	GetHandleVerifier [0x00007FF67DD182B2+55298]
	(No symbol) [0x00007FF67DC85E02]
	(No symbol) [0x00007FF67DB405AB]
	(No symbol) [0x00007FF67DB2E997]
	(No symbol) [0x00007FF67DB2E7A8]
	(No symbol) [0x00007FF67DB2CF0E]
	(No symbol) [0x00007FF67DB2D9CE]
	(No symbol) [0x00007FF67DB3A20F]
	(No symbol) [0x00007FF67DB4B551]
	(No symbol) [0x00007FF67DB4FC1A]
	(No symbol) [0x00007FF67DB2E032]
	(No symbol) [0x00007FF67DB4B42A]
	(No symbol) [0x00007FF67DBBAB0B]
	(No symbol) [0x00007FF67DBA1E83]
	(No symbol) [0x00007FF67DB7670A]
	(No symbol) [0x00007FF67DB77964]
	GetHandleVerifier [0x00007FF67E090AAB+3694587]
	GetHandleVerifier [0x00007FF67E0E728E+4048862]
	GetHandleVerifier [0x00007FF67E0DF173+4015811]
	GetHandleVerifier [0x00007FF67DDB47D6+695590]
	(No symbol) [0x00007FF67DC90CE8]
	(No symbol) [0x00007FF67DC8CF34]
	(No symbol) [0x00007FF67DC8D062]
	(No symbol) 

In [4]:
soup = BeautifulSoup(driver.page_source, 'html.parser')

In [5]:
trs = soup.select('#slidePage1 > #pointList > li')
len(trs)

5

In [6]:
trs.encoding = 'utf-8'
trs

[<li><img alt="세차지수 사진" src="../img/icon_idx_carwash.png?ver=20231201"/><h1>세차지수</h1><h2> 70</h2><p>비교적 걱정 없이 세차할 수 있어요</p><progress max="100" value="70"></progress></li>,
 <li><img alt="달리기지수 사진" src="../img/icon_idx_running.png?ver=20231201"/><h1>달리기지수</h1><h2> 55</h2><p>오늘은 달리기 보단 가벼운 산책 추천</p><progress max="100" value="55"></progress></li>,
 <li><img alt="빨래지수 사진" src="../img/icon_idx_washing.png?ver=20231201"/><h1>빨래지수</h1><h2> 54</h2><p>문을 닫고 실내에서 말리는 것을 추천</p><progress max="100" value="54"></progress></li>,
 <li><img alt="수면지수 사진" src="../img/icon_idx_sleep.png?ver=20231201"/><h1>수면지수</h1><h2> 30</h2><p>수면을 위해 적정 온습도를 유지하세요</p><progress max="100" value="30"></progress></li>,
 <li><img alt="우산지수 사진" src="../img/icon_idx_umb.png?ver=20231201"/><h1>우산지수</h1><h2> 10</h2><p>우산 없이 가볍게 외출하세요!</p><progress max="100" value="10"></progress></li>]

In [7]:
# 생활지수
tr = trs[0]
img = tr.select_one('img')['src']
item = tr.select_one('h1').get_text().strip().replace('.', '')
index = tr.select_one('h2').get_text().strip()
ment = tr.select_one('p').get_text().strip()
item, index, ment

('세차지수', '70', '비교적 걱정 없이 세차할 수 있어요')

In [8]:
data = []
from datetime import datetime, timedelta

for tr in trs:
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    img = tr.select_one('img')['src']
    item = tr.select_one('h1').get_text().strip().replace('.', '')
    index = tr.select_one('h2').get_text().strip()
    ment = tr.select_one('p').get_text().strip()


    url = 'https://www.kr-weathernews.com/mv3/html/lifeindex.html?region='
    res = requests.get(url)
    soup = BeautifulSoup(res.text, 'html.parser')

    data.append({ '이미지': img, '생활지수': item, '지수': index, '안내멘트': ment})

In [9]:
df = pd.DataFrame(data)
df

,이미지,생활지수,지수,안내멘트
0,../img/icon_idx_carwash.png?ver=20231201,세차지수,70,비교적 걱정 없이 세차할 수 있어요
1,../img/icon_idx_running.png?ver=20231201,달리기지수,55,오늘은 달리기 보단 가벼운 산책 추천
2,../img/icon_idx_washing.png?ver=20231201,빨래지수,54,문을 닫고 실내에서 말리는 것을 추천
3,../img/icon_idx_sleep.png?ver=20231201,수면지수,30,수면을 위해 적정 온습도를 유지하세요
4,../img/icon_idx_umb.png?ver=20231201,우산지수,10,우산 없이 가볍게 외출하세요!


In [3]:
#
url = 'https://www.kr-weathernews.com/mv3/html/lifeindex.html?region='
header = {'User-Agent' : 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36'}
res = requests.get(url, headers=header) 
driver = webdriver.Chrome()     # PATH 설정 할 것 (chromedriver.exe가 위치해있는곳을 path 설정해야함)
driver.get(url)
soup = BeautifulSoup(driver.page_source, 'html.parser')
trs2 = soup.select('#slidePage2 > #pointList > li')
len(trs2)

The chromedriver version (119.0.6045.105) detected in PATH at C:\Users\human\Downloads\chromedriver.exe might not be compatible with the detected chrome version (120.0.6099.71); currently, chromedriver 120.0.6099.71 is recommended for chrome 120.*, so it is advised to delete the driver in PATH and retry


3

In [4]:
#trs2.encoding = 'utf-8'
trs2

[<li><img alt="자외선지수 사진" src="../img/icon_life_uv.png?ver=20231201"/><h1>자외선지수</h1><h2> 0</h2><p>자외선 걱정 없는 날이에요.</p><progress max="100" value="25"></progress></li>,
 <li><img alt="감기지수 사진" src="../img/icon_life_cold.png?ver=20231201"/><h1>감기지수</h1><h2> 3</h2><p>노약자는 외부 활동을 되도록 삼가요</p><progress max="100" value="75"></progress></li>,
 <li><img alt="동파지수 사진" src="../img/icon_life_freez.png?ver=20231201"/><h1>동파지수</h1><h2> 25</h2><p>동파지수가 매우 낮아요</p><progress max="100" value="25"></progress></li>]

In [5]:
# 생활지수
soup = BeautifulSoup(driver.page_source, 'html.parser')
trs2 = soup.select('#slidePage2 > #pointList > li')
tr2 = trs2[0]
img = tr2.select_one('img')['src']
item = tr2.select_one('h1').get_text().strip().replace('.', '')
index = tr2.select_one('h2').get_text().strip()
ment = tr2.select_one('p').get_text().strip()
item, index, ment

('자외선지수', '0', '자외선 걱정 없는 날이에요.')

In [6]:
data = []
#url = 'https://www.kr-weathernews.com/mv3/html/lifeindex.html?region='
#driver = webdriver.Chrome()     # PATH 설정 할 것 (chromedriver.exe가 위치해있는곳을 path 설정해야함)
#driver.get(url)
#soup = BeautifulSoup(driver.page_source, 'html.parser')
#trs2 = soup.select('#slidePage2 > #pointList > li')
for tr2 in trs2:
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    img = tr2.select_one('img')['src']
    item = tr2.select_one('h1').get_text().strip().replace('.', '')
    index = tr2.select_one('h2').get_text().strip()
    ment = tr2.select_one('p').get_text().strip()


    url = 'https://www.kr-weathernews.com/mv3/html/lifeindex.html?region='
    res = requests.get(url)
    soup = BeautifulSoup(res.text, 'html.parser')

    data.append({ '이미지': img, '생활지수': item, '지수': index, '안내멘트': ment})

In [17]:
df = pd.DataFrame(data)
df

,이미지,생활지수,지수,안내멘트
0,../img/icon_idx_carwash.png?ver=20231201,세차지수,0,오늘은 세차하기에 최악의 날씨
1,../img/icon_idx_running.png?ver=20231201,달리기지수,17.5,오늘은 꼭 실내에서 운동하세요
2,../img/icon_idx_washing.png?ver=20231201,빨래지수,63,빨래하기에 적당한 날씨예요
3,../img/icon_idx_sleep.png?ver=20231201,수면지수,20,수면을 위해 적정 온습도를 유지하세요
4,../img/icon_idx_umb.png?ver=20231201,우산지수,60,우산 꼭 챙겨서 외출하세요
5,../img/icon_life_uv.png?ver=20231201,자외선지수,1,오늘은 안심하고 야외 활동하세요
6,../img/icon_life_cold.png?ver=20231201,감기지수,3,감기지수가 높으니 체온 유지는 필수
7,../img/icon_life_freez.png?ver=20231201,동파지수,25,오늘은 동파 걱정 끝!


In [8]:
# 데이터프레임을 텍스트 파일로 저장
df.to_csv('data/kweather.txt', sep='\t', index=False)

In [18]:
df.to_csv('data/kweather.csv', index=False)

In [19]:
data1 = []
data2 = []
url = 'https://www.kr-weathernews.com/mv3/html/lifeindex.html?region='
header = {'User-Agent' : 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36'}
res = requests.get(url, headers=header) 
driver = webdriver.Chrome()     # PATH 설정 할 것 (chromedriver.exe가 위치해있는곳을 path 설정해야함)
driver.get(url)
soup = BeautifulSoup(driver.page_source, 'html.parser')
trs = soup.select('#slidePage1 > #pointList > li')
trs2 = soup.select('#slidePage2 > #pointList > li')

for tr in trs:
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    img = tr.select_one('img')['src']
    item = tr.select_one('h1').get_text().strip().replace('.', '')
    index = tr.select_one('h2').get_text().strip()
    ment = tr.select_one('p').get_text().strip()


    url = 'https://www.kr-weathernews.com/mv3/html/lifeindex.html?region='
    res = requests.get(url)
    soup = BeautifulSoup(res.text, 'html.parser')

    data1.append({ '이미지': img, '생활지수': item, '지수': index, '안내멘트': ment})
    
for tr2 in trs2:
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    time.sleep(2)
    img = tr2.select_one('img')['src']
    item = tr2.select_one('h1').get_text().strip().replace('.', '')
    index = tr2.select_one('h2').get_text().strip()
    ment = tr2.select_one('p').get_text().strip()


    url = 'https://www.kr-weathernews.com/mv3/html/lifeindex.html?region='
    res = requests.get(url)
    soup = BeautifulSoup(res.text, 'html.parser')

    data2.append({ '이미지': img, '생활지수': item, '지수': index, '안내멘트': ment})

data = data1 + data2
df = pd.DataFrame(data)
df.to_csv('data/kweather.csv', index=False)

The chromedriver version (119.0.6045.105) detected in PATH at C:\Users\human\Downloads\chromedriver.exe might not be compatible with the detected chrome version (120.0.6099.71); currently, chromedriver 120.0.6099.71 is recommended for chrome 120.*, so it is advised to delete the driver in PATH and retry


In [16]:
data

[{'이미지': '../img/icon_idx_carwash.png?ver=20231201',
  '생활지수': '세차지수',
  '지수': '0',
  '안내멘트': '오늘은 세차하기에 최악의 날씨'},
 {'이미지': '../img/icon_idx_running.png?ver=20231201',
  '생활지수': '달리기지수',
  '지수': '17.5',
  '안내멘트': '오늘은 꼭 실내에서 운동하세요'},
 {'이미지': '../img/icon_idx_washing.png?ver=20231201',
  '생활지수': '빨래지수',
  '지수': '63',
  '안내멘트': '빨래하기에 적당한 날씨예요'},
 {'이미지': '../img/icon_idx_sleep.png?ver=20231201',
  '생활지수': '수면지수',
  '지수': '20',
  '안내멘트': '수면을 위해 적정 온습도를 유지하세요'},
 {'이미지': '../img/icon_idx_umb.png?ver=20231201',
  '생활지수': '우산지수',
  '지수': '60',
  '안내멘트': '우산 꼭 챙겨서 외출하세요'},
 {'이미지': '../img/icon_life_uv.png?ver=20231201',
  '생활지수': '자외선지수',
  '지수': '1',
  '안내멘트': '오늘은 안심하고 야외 활동하세요'},
 {'이미지': '../img/icon_life_cold.png?ver=20231201',
  '생활지수': '감기지수',
  '지수': '3',
  '안내멘트': '감기지수가 높으니 체온 유지는 필수'},
 {'이미지': '../img/icon_life_freez.png?ver=20231201',
  '생활지수': '동파지수',
  '지수': '25',
  '안내멘트': '오늘은 동파 걱정 끝!'}]